<a href="https://colab.research.google.com/github/Sobiii/Neural-Networks-CIFAR-10-Assignment/blob/main/Neural_Networks_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1) Reading CIFAR-10 and creating dataloaders (5%)



In [1]:
# Setting up google drive 

from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
import sys
sys.path.append('/content/gdrive/MyDrive/Colab Notebooks')

Mounted at /content/gdrive


The CIFAR-10 python dataset has been downloaded from the University of Toronto website.

In [87]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn

# Defining data augmentation transformations

transform_train = transforms.Compose(
    [transforms.RandomHorizontalFlip(),
     transforms.RandomCrop(32, padding=4),
     transforms.RandomRotation(15),
     transforms.RandomVerticalFlip(),
     transforms.RandomGrayscale(p=0.1),
     transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Defining test data transformations

transform_test = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Loading the CIFAR-10 dataset

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_train)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform_test)

# Creating dataloaders

trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=False, num_workers=2)

# Defining class labels

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Accessing labels from dataloaders

for images, labels in trainloader:
    print(images.shape)  # shape of images batch 
    print(labels.shape)  # shape of labels batch 
    break  


Files already downloaded and verified
Files already downloaded and verified
torch.Size([64, 3, 32, 32])
torch.Size([64])


# 2) Creating the model (40%)

[Note to self - these hyper parameter settings are important for fine tuning the model]

In [88]:
import torch
import torch.nn as nn

class MyModel(nn.Module):
    def __init__(self, in_channels, out_channels, num_blocks, num_conv_layers, num_classes, hidden_layers, hidden_units):
        super(MyModel, self).__init__()
        self.mlp_layer = nn.Linear(in_channels, out_channels)  # MLP layer to compute 'a'
        self.conv_layers = nn.ModuleList()
        for _ in range(num_blocks):
            for _ in range(num_conv_layers):
                conv_layer = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
                self.conv_layers.append(conv_layer)
            in_channels = out_channels  # Updating the number of input channels for the next block
        self.avg_pool = nn.AdaptiveAvgPool2d(1)  # SpatialAveragePooling

        # The MLP Classifier

        self.classifier = nn.Sequential()
        if hidden_layers > 0:
            self.classifier.add_module('hidden_layer_0', nn.Linear(out_channels, hidden_units))
            self.classifier.add_module('relu_0', nn.ReLU(inplace=True))
            for i in range(1, hidden_layers):
                self.classifier.add_module(f'hidden_layer_{i}', nn.Linear(hidden_units, hidden_units))
                self.classifier.add_module(f'relu_{i}', nn.ReLU(inplace=True))
            self.classifier.add_module('output_layer', nn.Linear(hidden_units, num_classes))
        else:
            self.classifier.add_module('output_layer', nn.Linear(out_channels, num_classes))

    def forward(self, x):
        a = self.mlp_layer(self.avg_pool(x).squeeze())  # Prediciting 'a' using the MLP layer
        a_list = []
        for conv_layer in self.conv_layers:
            x = conv_layer(x)
            a_list.append(x)
        O = torch.sum(torch.stack(a_list, dim=1), dim=1)  # Combine outputs using sum along the stack dimension
        f = self.avg_pool(O).squeeze()  # Calculating 'f' using SpatialAveragePooling
        output = self.classifier(f)  # Passing 'f' to the MLP classifier
        return output, a_list

# Example model parameters:

in_channels = 3  # Number of input channels (3 for RGB images)
out_channels = 64  # Number of output channels [Can be fined tuned for model performance]
num_blocks = 3  # Number of blocks in the backbone [Can be fined tuned for model performance]
num_conv_layers = 2  # Number of convolutional layers per block (K) [Can be fined tuned for model performance]
num_classes = 10  # Number of output classes for classification

hidden_layers = 2  # Number of hidden layers in the MLP classifier [Can be fined tuned for model performance]
hidden_units = 256  # Number of hidden units in each hidden layer of the MLP classifier [Can be fined tuned for model performance]


# Printing the models architecture

model = MyModel(in_channels, out_channels, num_blocks, num_conv_layers, num_classes, hidden_layers, hidden_units)
print(model)


MyModel(
  (mlp_layer): Linear(in_features=3, out_features=64, bias=True)
  (conv_layers): ModuleList(
    (0-5): 6 x Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (avg_pool): AdaptiveAvgPool2d(output_size=1)
  (classifier): Sequential(
    (hidden_layer_0): Linear(in_features=64, out_features=256, bias=True)
    (relu_0): ReLU(inplace=True)
    (hidden_layer_1): Linear(in_features=256, out_features=256, bias=True)
    (relu_1): ReLU(inplace=True)
    (output_layer): Linear(in_features=256, out_features=10, bias=True)
  )
)


#3) Creating the loss and optimiser (5%)

#4) Training script (30%)

#5) Final model accuracy (5%)